### DEMO 3: OPTIMIZACIÓN DE NECESIDADES DE STOCK
HACIENDO EL MODELO DINAMICO A LOS CAMBIOS EN LOS DATOS (PARTE 1)

In [1]:
from docplex.mp.model import Model

In [2]:
# CARGA DE DATOS

producto = ['M', 'S', 'B']
categoria = ['MESA', 'SILLA', 'BIBLIOTECA']
margen = [10, 12, 30]
peso = [4, 4, 4]
stk_seg = [100, 110, 120]
stk = [12, 15, 20]
stk_max = [200, 300, 300]
dem = [150, 150, 150]
costo = [5.5, 5.5, 5.5]
sup = [4, 4, 4]
sup_max = 4000
peso_max = 4000
inv_max = 12000

In [ ]:
# (1) MAXIMIZAR GANANCIAS - DEFINICIÓN DE VARIABLES DE DECISIÓN Y FUNCIONAL

modelo = Model()

Q = modelo.integer_var_list(producto, name='Q')

modelo.maximize(modelo.sum(Q[i] * margen[i] 
                           for i in range(len(producto)))) # AHORA EL FUNCIONAL RESPONDE AUTOMATICAMENTE A LOS CAMBIOS EN LOS DATOS

# (2) LIMITACION DE SUPERFICIE PARA BIBLIOTECAS

modelo.add_constraint(sup[2]*stk[2] + sup[2]*Q[2] <= sup_max)

# (3) RESTRICCIONES DE STOCK DE SEGURIDAD

modelo.add_constraint(stk[0] + Q[0] >= stk_seg[0])
modelo.add_constraint(stk[1] + Q[1] >= stk_seg[1])
modelo.add_constraint(stk[2] + Q[2] >= stk_seg[2])

### (4) RESTRICCIONES DE MÁXIMO STOCK 

modelo.add_constraint(stk[0] + Q[0] <= stk_max[0])
modelo.add_constraint(stk[1] + Q[1] <= stk_max[1])
modelo.add_constraint(stk[2] + Q[2] <= stk_max[2])

# (5) RESTRICCIONES DE PESO (MESAS Y SILLAS)

modelo.add_constraint(peso[0]*Q[0] + peso[1]*Q[1] <= peso_max)

# (6) RESTRICCIONES INVERSIÓN

modelo.add_constraint(modelo.sum(costo[i]*Q[i] for i in range(len(producto))  ) 
                      <= inv_max) # AHORA ESTA RESTRICCIÓN RESPONDE AUTOMATICAMENTE A LOS CAMBIOS EN LOS DATOS

# (7) RESTRICCIONES SEGÚN ESTIMACIÓN DE DEMANDA

modelo.add_constraint(stk[0] + Q[0]  <= 1.1*dem[0])
modelo.add_constraint(stk[1] + Q[1]  <= 1.1*dem[1])
modelo.add_constraint(stk[2] + Q[2]  <= 1.1*dem[2])

# RESOLUCIÓN DEL MODELO

s = modelo.solve()

# IMPRESIÓN DE RESULTADOS

print("MESAS          "," = ", s.get_value(Q[0]))
print("SILLAS         "," = ", s.get_value(Q[1]))
print("BIBLIOTECAS    "," = ", s.get_value(Q[2]))
print("MARGEN TOTAL: $", modelo.solution.get_objective_value())

MESAS            =  153.0
SILLAS           =  150.0
BIBLIOTECAS      =  145.0
MARGEN TOTAL: $ 7680.0


In [4]:
modelo.print_information()
print(modelo.export_as_lp_string())

Model: docplex_model1
 - number of variables: 3
   - binary=0, integer=3, continuous=0
 - number of constraints: 12
   - linear=12
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Maximize
 obj: 10 Q_M + 12 Q_S + 30 Q_B
Subject To
 c1: 4 Q_B <= 3920
 c2: Q_M >= 88
 c3: Q_S >= 95
 c4: Q_B >= 100
 c5: Q_M <= 188
 c6: Q_S <= 285
 c7: Q_B <= 280
 c8: 4 Q_M + 4 Q_S <= 4000
 c9: 5.500000000000 Q_M + 5.500000000000 Q_S + 5.500000000000 Q_B <= 12000
 c10: Q_M <= 153
 c11: Q_S <= 150
 c12: Q_B <= 145

Bounds

Generals
 Q_M Q_S Q_B
End

